In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
#from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
#from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from xgboost import XGBRegressor
import lightgbm as lgb
from catboost import CatBoostClassifier
#from sklearn.linear_model import LinearRegression
import optuna

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# check on the panda version and its dependencies
# i run this from time to time to ensure all is up to date
pd.__version__
#pd.show_versions()

In [ ]:
# kfolds-5/train_folds_5.csv is an output from https://www.kaggle.com/code/andrewnuk/kfolds-5

df_train = pd.read_csv('/kaggle/input/kfolds-5/train_folds_5.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')
df_sampleSubmission = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/sample_submission.csv')

In [ ]:
# detect number of folds
fold_no = df_train['kfolds'].max() +1
fold_no

In [ ]:
df_train.head()

In [ ]:
# looking at f_27

In [ ]:
# all strings in f_27 are length 10 in train and test data

# df_train['f_27_len'] = df_train['f_27'].astype(str).map(len)
# df_test['f_27_len'] = df_test['f_27'].astype(str).map(len)
# df_train['f_27_len'].unique().tolist(), df_test['f_27_len'].unique().tolist()

In [ ]:
# label encode

f27_col_list = []
for i in range(1,11):
    df_train['f_27_' + str(i)] = df_train['f_27'].str.split('',expand=True)[i]
    df_test['f_27_' + str(i)] = df_test['f_27'].str.split('',expand=True)[i]
    f27_col_list.append('f_27_' + str(i))

In [ ]:
df_train

In [ ]:
f27_col_list

In [ ]:
for col in f27_col_list:
    df_train[col] = LabelEncoder().fit_transform(df_train[col])
    df_test[col] = LabelEncoder().fit_transform(df_test[col])

In [ ]:
unique_f27=[]
unique_f27_test=[]


for item in df_train['f_27']:
    unique_f27.append(len(set(item)))
    
for item in df_test['f_27']:
    unique_f27_test.append(len(set(item)))
    

df_train['unique_f27'] = unique_f27
df_test['unique_f27'] = unique_f27_test

In [ ]:
# more label encoding of f_29 and f_30

df_train['f_30_0'] = np.where(df_train['f_30'] == 0, 1, 0)
df_train['f_30_1'] = np.where(df_train['f_30'] == 1, 1, 0) 
df_train['f_30_2'] = np.where(df_train['f_30'] == 2, 1, 0) 
df_test['f_30_0'] = np.where(df_test['f_30'] == 0, 1, 0)
df_test['f_30_1'] = np.where(df_test['f_30'] == 1, 1, 0) 
df_test['f_30_2'] = np.where(df_test['f_30'] == 2, 1, 0) 

df_train['f_29_0'] = np.where(df_train['f_29'] == 0, 1, 0)
df_train['f_29_1'] = np.where(df_train['f_29'] == 1, 1, 0) 
df_test['f_29_0'] = np.where(df_test['f_29'] == 0, 1, 0)
df_test['f_29_1'] = np.where(df_test['f_29'] == 1, 1, 0) 


In [ ]:
f29_col_list=['f_29_0','f_29_1']

for col in f29_col_list:
    df_train[col] = LabelEncoder().fit_transform(df_train[col])
    df_test[col] = LabelEncoder().fit_transform(df_test[col])

In [ ]:
f30_col_list=['f_30_0','f_30_1','f_30_2']

for col in f30_col_list:
    df_train[col] = LabelEncoder().fit_transform(df_train[col])
    df_test[col] = LabelEncoder().fit_transform(df_test[col])

In [ ]:
continuous_feat = ['f_00', 'f_01', 'f_02', 'f_03', 'f_04', 'f_05', 'f_06', 'f_19',
                   'f_20', 'f_21', 'f_22', 'f_23', 'f_24', 'f_25', 'f_26', 'f_28']

def stat_features(df, cols = continuous_feat):
    '''
    Calculate aggregated features across the selected continuous columns
    
    '''
    # Base statistical features.
    df['f_sum']  = df[continuous_feat].sum(axis=1)
    df['f_min']  = df[continuous_feat].min(axis=1)
    df['f_max']  = df[continuous_feat].max(axis=1)
    df['f_std']  = df[continuous_feat].std(axis=1)    
    df['f_mad']  = df[continuous_feat].mad(axis=1)
    df['f_mean'] = df[continuous_feat].mean(axis=1)
    df['f_kurt'] = df[continuous_feat].kurt(axis=1)

    # Extra statistical features.
    df['f_prod'] = df[continuous_feat].prod(axis=1)
    df['f_range'] = df[continuous_feat].max(axis=1) - df[continuous_feat].min(axis=1)
    df['f_count_pos']  = df[df[continuous_feat].gt(0)].count(axis=1)
    df['f_count_neg']  = df[df[continuous_feat].lt(0)].count(axis=1)

    return df

In [ ]:
df_train = stat_features(df_train, continuous_feat)
df_test = stat_features(df_test, continuous_feat)

In [ ]:
df_train

In [ ]:
df_test

In [ ]:
df_train.columns, df_test.columns

In [ ]:
# rename the nominal columns
# list of nominal columns will be where there are less than 20 unique numers

nom_cols = []

for i in range(len(df_train.columns)):
    if df_train[df_train.columns[i]].nunique() < 30:
        nom_cols.append(df_train.columns[i])

# remove 'target', 'kfolds'
nom_cols.remove('target')
nom_cols.remove('kfolds')

print(nom_cols)

In [ ]:
# put the rest of the columns in another list to scale
# and remove f_27

all_cols = df_train.columns.tolist()

rest_cols = list(set(all_cols) - set(nom_cols))
rest_cols.remove('target')
rest_cols.remove('id')
rest_cols.remove('kfolds')
rest_cols.remove('f_27')


print(rest_cols)

In [ ]:
# scale the non integer columns

scaler = RobustScaler()
df_train[rest_cols] = scaler.fit_transform(df_train[rest_cols])
df_test[rest_cols] = scaler.transform(df_test[rest_cols]) 

In [ ]:
df_train.head()

In [ ]:
useful_features = [c for c in all_cols if c not in ('id','target','kfolds','f_27','f_29','f_30')]
df_test = df_test[useful_features]
print(useful_features)

In [ ]:
def run(trial):
    fold = 0 # lets run over 1 fold only, not all the folds
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-8, 100.0)
    subsample = trial.suggest_float("subsample", 0.1, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    max_depth = trial.suggest_int("max_depth", 1, 7)

    xtrain = df_train[df_train['kfolds'] != fold].reset_index(drop=True)
    xvalid = df_train[df_train['kfolds'] == fold].reset_index(drop=True)

    ytrain = xtrain.target
    yvalid = xvalid.target

    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]

#     ordinal_encoder = preprocessing.OrdinalEncoder()
#     xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
#     xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])


# do i still need to set use_label_encoder='false'?

    model = XGBRegressor(
        random_state=0,
        tree_method="hist",
        eval_metric='auc',
        objective='binary:logistic',
        use_label_encoder='false',
        n_jobs=-1,
        n_estimators=10000,
        learning_rate=learning_rate,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        max_depth=max_depth,
    )
    model.fit(xtrain, ytrain, early_stopping_rounds=1000, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict(xvalid)
    roc = roc_auc_score(yvalid, preds_valid)
    return roc

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(run, n_trials=10)

In [ ]:
study.best_params

In [ ]:
# 12/5/2022

# {'learning_rate': 0.09096036362714063,
#  'reg_lambda': 6.799218986620073e-05,
#  'reg_alpha': 0.0003145752508772316,
#  'subsample': 0.8750417933515143,
#  'colsample_bytree': 0.8118107556164477,
#  'max_depth': 7}

# 0.9750681033246448

In [ ]:
# 14/5/22

# {'learning_rate': 0.23671532343035417,
#  'reg_lambda': 0.06524978666003266,
#  'reg_alpha': 10.995834532927617,
#  'subsample': 0.667602746900821,
#  'colsample_bytree': 0.2839507013298166,
#  'max_depth': 7}

# 0.9630306483491208

In [ ]:
# 21/5/22

# {'learning_rate': 0.05577059893375157,
#  'reg_lambda': 4.7271008102542467e-08,
#  'reg_alpha': 2.172737119173752,
#  'subsample': 0.5107075212321727,
#  'colsample_bytree': 0.8002715671084749,
#  'max_depth': 7}

# 0.9897120909794579

In [ ]:
# 31/5/22

# {'learning_rate': 0.15343501287004324,
#  'reg_lambda': 9.738544085707843e-07,
#  'reg_alpha': 0.0007894750581813947,
#  'subsample': 0.5602446612972992,
#  'colsample_bytree': 0.49834681824944793,
#  'max_depth': 5}

# 0.9857334297889333

In [ ]:
# 24/5/2022

# {'learning_rate': 0.06505977954394805,
#  'reg_lambda': 0.006270100299831277,
#  'reg_alpha': 1.2169773130333711e-05,
#  'subsample': 0.9796263401134638,
#  'colsample_bytree': 0.7486916086061227,
#  'max_depth': 6}

# 0.9901056372022148

In [ ]:
# kfolds-5/train_folds_5.csv is an output from https://www.kaggle.com/code/andrewnuk/kfolds-5

df_train = pd.read_csv('/kaggle/input/kfolds-5/train_folds_5.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')
df_sampleSubmission = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/sample_submission.csv')

In [ ]:
# label encode

f27_col_list = []
for i in range(1,11):
    df_train['f_27_' + str(i)] = df_train['f_27'].str.split('',expand=True)[i]
    df_test['f_27_' + str(i)] = df_test['f_27'].str.split('',expand=True)[i]
    f27_col_list.append('f_27_' + str(i))

In [ ]:
df_train

In [ ]:
f27_col_list

In [ ]:
for col in f27_col_list:
    df_train[col] = LabelEncoder().fit_transform(df_train[col])
    df_test[col] = LabelEncoder().fit_transform(df_test[col])

In [ ]:
df_train

In [ ]:
unique_f27=[]
unique_f27_test=[]


for item in df_train['f_27']:
    unique_f27.append(len(set(item)))
    
for item in df_test['f_27']:
    unique_f27_test.append(len(set(item)))
    

df_train['unique_f27'] = unique_f27
df_test['unique_f27'] = unique_f27_test

In [ ]:
# more label encoding of f_29 and f_30

df_train['f_30_0'] = np.where(df_train['f_30'] == 0, 1, 0)
df_train['f_30_1'] = np.where(df_train['f_30'] == 1, 1, 0) 
df_train['f_30_2'] = np.where(df_train['f_30'] == 2, 1, 0) 
df_test['f_30_0'] = np.where(df_test['f_30'] == 0, 1, 0)
df_test['f_30_1'] = np.where(df_test['f_30'] == 1, 1, 0) 
df_test['f_30_2'] = np.where(df_test['f_30'] == 2, 1, 0) 

df_train['f_29_0'] = np.where(df_train['f_29'] == 0, 1, 0)
df_train['f_29_1'] = np.where(df_train['f_29'] == 1, 1, 0) 
df_test['f_29_0'] = np.where(df_test['f_29'] == 0, 1, 0)
df_test['f_29_1'] = np.where(df_test['f_29'] == 1, 1, 0) 


In [ ]:
f29_col_list=['f_29_0','f_29_1']

for col in f29_col_list:
    df_train[col] = LabelEncoder().fit_transform(df_train[col])
    df_test[col] = LabelEncoder().fit_transform(df_test[col])

In [ ]:
f30_col_list=['f_30_0','f_30_1','f_30_2']

for col in f30_col_list:
    df_train[col] = LabelEncoder().fit_transform(df_train[col])
    df_test[col] = LabelEncoder().fit_transform(df_test[col])

In [ ]:
continuous_feat = ['f_00', 'f_01', 'f_02', 'f_03', 'f_04', 'f_05', 'f_06', 'f_19',
                   'f_20', 'f_21', 'f_22', 'f_23', 'f_24', 'f_25', 'f_26', 'f_28']

def stat_features(df, cols = continuous_feat):
    '''
    Calculate aggregated features across the selected continuous columns
    
    '''
    # Base statistical features.
    df['f_sum']  = df[continuous_feat].sum(axis=1)
    df['f_min']  = df[continuous_feat].min(axis=1)
    df['f_max']  = df[continuous_feat].max(axis=1)
    df['f_std']  = df[continuous_feat].std(axis=1)    
    df['f_mad']  = df[continuous_feat].mad(axis=1)
    df['f_mean'] = df[continuous_feat].mean(axis=1)
    df['f_kurt'] = df[continuous_feat].kurt(axis=1)

    # Extra statistical features.
    df['f_prod'] = df[continuous_feat].prod(axis=1)
    df['f_range'] = df[continuous_feat].max(axis=1) - df[continuous_feat].min(axis=1)
    df['f_count_pos']  = df[df[continuous_feat].gt(0)].count(axis=1)
    df['f_count_neg']  = df[df[continuous_feat].lt(0)].count(axis=1)

    return df

In [ ]:
df_train = stat_features(df_train, continuous_feat)
df_test = stat_features(df_test, continuous_feat)

In [ ]:
# rename the nominal columns
# list of nominal columns will be where there are less than 20 unique numers

nom_cols = []

for i in range(len(df_train.columns)):
    if df_train[df_train.columns[i]].nunique() < 30:
        nom_cols.append(df_train.columns[i])

# remove 'target'
nom_cols.remove('target')
nom_cols.remove('kfolds')

print(nom_cols)

In [ ]:
# put the rest of the columns in another list to scale
# and remove f_27

all_cols = df_train.columns.tolist()

rest_cols = list(set(all_cols) - set(nom_cols))
rest_cols.remove('target')
rest_cols.remove('id')
rest_cols.remove('kfolds')
rest_cols.remove('f_27')

print(rest_cols)

In [ ]:
# scale the non integer columns

scaler = RobustScaler()
df_train[rest_cols] = scaler.fit_transform(df_train[rest_cols])
df_test[rest_cols] = scaler.transform(df_test[rest_cols]) 

In [ ]:
df_train.head()

In [ ]:
useful_features = [c for c in all_cols if c not in ('id','target','kfolds','f_27','f_29','f_30')]
df_test = df_test[useful_features]
print(useful_features)

In [ ]:
def run_cb(trial):
    fold = 0 # lets run over 1 fold only, not all the folds
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
    l2_leaf_reg = trial.suggest_float("l2_leaf_reg", 1e-2, 10)
    min_data_in_leaf = trial.suggest_int("min_data_in_leaf", 1, 250)
    depth = trial.suggest_int("depth", 1, 7)
    leaf_estimation_iterations=trial.suggest_int("leaf_estimation_iterations", 1 , 10) 
    subsample=trial.suggest_float("subsample", 1e-2, 1, log=True)

    xtrain = df_train[df_train['kfolds'] != fold].reset_index(drop=True)
    xvalid = df_train[df_train['kfolds'] == fold].reset_index(drop=True)

    ytrain = xtrain.target
    yvalid = xvalid.target

    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]

#     ordinal_encoder = preprocessing.OrdinalEncoder()
#     xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
#     xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])


# do i still need to set use_label_encoder='false'?

    model = CatBoostClassifier(
        random_state=0,
        loss_function="CrossEntropy",
        eval_metric='AUC',
        bootstrap_type='Bernoulli',
        n_estimators=10000,
        learning_rate=learning_rate,
        l2_leaf_reg=l2_leaf_reg,
        min_data_in_leaf=min_data_in_leaf,
        depth=depth,
        leaf_estimation_iterations=leaf_estimation_iterations,
        subsample=subsample,
    )
    model.fit(xtrain, ytrain, early_stopping_rounds=1000, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict_proba(xvalid)[:, -1]
    roc = roc_auc_score(yvalid, preds_valid)
    return roc

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(run_cb, n_trials=10)

In [ ]:
study.best_params

In [ ]:
# 12/5/2022

# {'learning_rate': 0.08684553746722419,
#  'l2_leaf_reg': 8.402827192199158,
#  'min_data_in_leaf': 217,
#  'depth': 7,
#  'leaf_estimation_iterations': 5,
#  'subsample': 0.8081438849362292}

# 0.9749720824251269

In [ ]:
# 14/5/2022

# {'learning_rate': 0.1407708903432696,
#  'l2_leaf_reg': 0.2639641855064276,
#  'min_data_in_leaf': 138,
#  'depth': 5,
#  'leaf_estimation_iterations': 4,
#  'subsample': 0.46256960096507777}

# 0.9690867067767097

In [ ]:
# 21/5/2022

# {'learning_rate': 0.0968013568491325,
#  'l2_leaf_reg': 8.24170046519059,
#  'min_data_in_leaf': 59,
#  'depth': 5,
#  'leaf_estimation_iterations': 6,
#  'subsample': 0.38582133041619293}

# 0.9880136289417993

In [ ]:
# 23/5/2022

# {'learning_rate': 0.1807371501496165,
#  'l2_leaf_reg': 1.0128782373400498,
#  'min_data_in_leaf': 83,
#  'depth': 7,
#  'leaf_estimation_iterations': 2,
#  'subsample': 0.7995161363350368}

# 0.9904511189593687

In [ ]:
# 24/5/2022

# {'learning_rate': 0.05925309541874935,
#  'l2_leaf_reg': 5.014610271661365,
#  'min_data_in_leaf': 226,
#  'depth': 7,
#  'leaf_estimation_iterations': 10,
#  'subsample': 0.09712001963001703}

# 0.9870812076240453

In [ ]:
# kfolds-5/train_folds_5.csv is an output from https://www.kaggle.com/code/andrewnuk/kfolds-5

df_train = pd.read_csv('/kaggle/input/kfolds-5/train_folds_5.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')
df_sampleSubmission = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/sample_submission.csv')

In [ ]:
# label encode

f27_col_list = []
for i in range(1,11):
    df_train['f_27_' + str(i)] = df_train['f_27'].str.split('',expand=True)[i]
    df_test['f_27_' + str(i)] = df_test['f_27'].str.split('',expand=True)[i]
    f27_col_list.append('f_27_' + str(i))

In [ ]:
df_train

In [ ]:
f27_col_list

In [ ]:
for col in f27_col_list:
    df_train[col] = LabelEncoder().fit_transform(df_train[col])
    df_test[col] = LabelEncoder().fit_transform(df_test[col])

In [ ]:
df_train

In [ ]:
unique_f27=[]
unique_f27_test=[]


for item in df_train['f_27']:
    unique_f27.append(len(set(item)))
    
for item in df_test['f_27']:
    unique_f27_test.append(len(set(item)))
    

df_train['unique_f27'] = unique_f27
df_test['unique_f27'] = unique_f27_test

In [ ]:
# more label encoding of f_29 and f_30

df_train['f_30_0'] = np.where(df_train['f_30'] == 0, 1, 0)
df_train['f_30_1'] = np.where(df_train['f_30'] == 1, 1, 0) 
df_train['f_30_2'] = np.where(df_train['f_30'] == 2, 1, 0) 
df_test['f_30_0'] = np.where(df_test['f_30'] == 0, 1, 0)
df_test['f_30_1'] = np.where(df_test['f_30'] == 1, 1, 0) 
df_test['f_30_2'] = np.where(df_test['f_30'] == 2, 1, 0) 

df_train['f_29_0'] = np.where(df_train['f_29'] == 0, 1, 0)
df_train['f_29_1'] = np.where(df_train['f_29'] == 1, 1, 0) 
df_test['f_29_0'] = np.where(df_test['f_29'] == 0, 1, 0)
df_test['f_29_1'] = np.where(df_test['f_29'] == 1, 1, 0) 


In [ ]:
f29_col_list=['f_29_0','f_29_1']

for col in f29_col_list:
    df_train[col] = LabelEncoder().fit_transform(df_train[col])
    df_test[col] = LabelEncoder().fit_transform(df_test[col])

In [ ]:
f30_col_list=['f_30_0','f_30_1','f_30_2']

for col in f30_col_list:
    df_train[col] = LabelEncoder().fit_transform(df_train[col])
    df_test[col] = LabelEncoder().fit_transform(df_test[col])

In [ ]:
continuous_feat = ['f_00', 'f_01', 'f_02', 'f_03', 'f_04', 'f_05', 'f_06', 'f_19',
                   'f_20', 'f_21', 'f_22', 'f_23', 'f_24', 'f_25', 'f_26', 'f_28']

def stat_features(df, cols = continuous_feat):
    '''
    Calculate aggregated features across the selected continuous columns
    
    '''
    # Base statistical features.
    df['f_sum']  = df[continuous_feat].sum(axis=1)
    df['f_min']  = df[continuous_feat].min(axis=1)
    df['f_max']  = df[continuous_feat].max(axis=1)
    df['f_std']  = df[continuous_feat].std(axis=1)    
    df['f_mad']  = df[continuous_feat].mad(axis=1)
    df['f_mean'] = df[continuous_feat].mean(axis=1)
    df['f_kurt'] = df[continuous_feat].kurt(axis=1)

    # Extra statistical features.
    df['f_prod'] = df[continuous_feat].prod(axis=1)
    df['f_range'] = df[continuous_feat].max(axis=1) - df[continuous_feat].min(axis=1)
    df['f_count_pos']  = df[df[continuous_feat].gt(0)].count(axis=1)
    df['f_count_neg']  = df[df[continuous_feat].lt(0)].count(axis=1)

    return df

In [ ]:
df_train = stat_features(df_train, continuous_feat)
df_test = stat_features(df_test, continuous_feat)

In [ ]:
# rename the nominal columns
# list of nominal columns will be where there are less than 20 unique numers

nom_cols = []

for i in range(len(df_train.columns)):
    if df_train[df_train.columns[i]].nunique() < 30:
        nom_cols.append(df_train.columns[i])
    
# remove 'target'
nom_cols.remove('target')
nom_cols.remove('kfolds')

print(nom_cols)

In [ ]:
# put the rest of the columns in another list to scale
# and remove f_27

all_cols = df_train.columns.tolist()

rest_cols = list(set(all_cols) - set(nom_cols))
rest_cols.remove('target')
rest_cols.remove('id')
rest_cols.remove('kfolds')
rest_cols.remove('f_27')

print(rest_cols)

In [ ]:
# scale the non integer columns

scaler = RobustScaler()
df_train[rest_cols] = scaler.fit_transform(df_train[rest_cols])
df_test[rest_cols] = scaler.transform(df_test[rest_cols]) 

In [ ]:
df_train.head()

In [ ]:
useful_features = [c for c in all_cols if c not in ('id','target','kfolds','f_27','f_29','f_30')]
df_test = df_test[useful_features]
print(useful_features)

In [ ]:
def run_lgb(trial):
    fold = 0 # lets run over 1 fold only, not all the folds
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-8, 100.0)    
    subsample = trial.suggest_float("subsample", 0.1, 1.0)    
    subsample_freq = trial.suggest_int("subsample_freq", 1, 10)    
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 350)
    min_child_samples = trial.suggest_int("min_child_samples", 1, 250)


    xtrain = df_train[df_train['kfolds'] != fold].reset_index(drop=True)
    xvalid = df_train[df_train['kfolds'] == fold].reset_index(drop=True)

    ytrain = xtrain.target
    yvalid = xvalid.target

    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]

#     ordinal_encoder = preprocessing.OrdinalEncoder()
#     xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
#     xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])


# do i still need to set use_label_encoder='false'?

    model = lgb.LGBMClassifier(
        random_state=0,
        objective='binary',
        metric='auc',
        n_estimators=20000,
        learning_rate =learning_rate,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        subsample=subsample,
        subsample_freq=subsample_freq,
        colsample_bytree=colsample_bytree,
        min_child_weight=min_child_weight,
        min_child_samples=min_child_samples,
)
    
    model.fit(xtrain, ytrain, early_stopping_rounds=1000, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict_proba(xvalid)[:, -1]
    roc = roc_auc_score(yvalid, preds_valid)
    return roc

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(run_lgb, n_trials=10)

In [ ]:
study.best_params

In [ ]:
# 12/5/2022

# {'learning_rate': 0.0406731843696733,
#  'reg_lambda': 0.005386409050131694,
#  'reg_alpha': 3.804736083347904,
#  'subsample': 0.3263166862949025,
#  'subsample_freq': 2,
#  'colsample_bytree': 0.5089364252917477,
#  'min_child_weight': 17,
#  'min_child_samples': 168}

# 0.9677971386266209

In [ ]:
# 14/5/2022

# {'learning_rate': 0.019415225203461502,
#  'reg_lambda': 0.013475574062427073,
#  'reg_alpha': 4.858051878575738e-07,
#  'subsample': 0.8139094925249263,
#  'subsample_freq': 9,
#  'colsample_bytree': 0.5345304315196567,
#  'min_child_weight': 13,
#  'min_child_samples': 154}

# 0.9719534351674166

In [ ]:
# 21/5/2022

# {'learning_rate': 0.07247892295165916,
#  'reg_lambda': 3.205733096464452e-08,
#  'reg_alpha': 4.066866969871943e-05,
#  'subsample': 0.5049012303165791,
#  'subsample_freq': 6,
#  'colsample_bytree': 0.4489171977419022,
#  'min_child_weight': 7,
#  'min_child_samples': 165}

# 0.9873959812038946

In [ ]:
# 23/5/2022

# {'learning_rate': 0.0422455169968268,
#  'reg_lambda': 0.003410330040463861,
#  'reg_alpha': 0.11717078057504875,
#  'subsample': 0.4788217383699864,
#  'subsample_freq': 3,
#  'colsample_bytree': 0.42337949075443315,
#  'min_child_weight': 36,
#  'min_child_samples': 171}

# 0.9860704103972513

In [ ]:
# 24/5/2022

# {'learning_rate': 0.011018952972568469,
#  'reg_lambda': 0.002791795766942212,
#  'reg_alpha': 0.07064985771325745,
#  'subsample': 0.7201769968468187,
#  'subsample_freq': 3,
#  'colsample_bytree': 0.8876569537932815,
#  'min_child_weight': 23,
#  'min_child_samples': 101}

# 0.9891374260793635